In [41]:
# import 
import pandas as pd
import numpy as np
import folium
import json

In [42]:
# 스타벅스 매장리스트
sbuck_df = pd.read_excel('./seoul_starbucks.xlsx', header=0)
sbuck_df.head()

,매장명,위도,경도,매장타입,주소
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동)
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동)
2,신사역성일빌딩,37.513931,127.020606,general,서울특별시 강남구 강남대로 584 (논현동)
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동)
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동)


In [43]:
sbuck_df['주소'][599].split()[1]

'중랑구'

In [44]:
gu_lsit = []
for item in sbuck_df['주소']:
    gu = item.split()[1]
    gu_lsit.append(gu)

sbuck_df['구'] = gu_lsit
sbuck_df.head(100)

,매장명,위도,경도,매장타입,주소,구
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),강남구
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),강남구
2,신사역성일빌딩,37.513931,127.020606,general,서울특별시 강남구 강남대로 584 (논현동),강남구
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),강남구
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),강남구
...,...,...,...,...,...,...
95,마곡역,37.559611,126.826298,general,서울특별시 강서구 공항대로 168 (마곡동),강서구
96,우장산역,37.549501,126.835984,general,서울특별시 강서구 강서로 267 (내발산동),강서구
97,마곡사이언스타워R,37.560526,126.833887,reserve,서울특별시 강서구 마곡동로 62 (마곡동),강서구
98,방화DT,37.574339,126.816415,generalDT,서울특별시 강서구 방화대로 375 (방화동),강서구


#### 서울시 행정구역별 위경도 전처리

- 대한민국 행정구역 위경도엑셀파일에서 서울만 필터링

In [45]:
seoul_loc_df = pd.read_excel('./대한민국_행정구역_위경도.xlsx')
seoul_loc_df.head()

,docity,do,city,longitude,latitude
0,NaN,강원,강릉시,128.878497,37.749136
1,NaN,강원,고성군,128.470164,38.377961
2,NaN,강원,동해시,129.116633,37.521931
3,NaN,강원,삼척시,129.167489,37.447086
4,NaN,강원,속초시,128.594167,38.204275


In [88]:
seoul_gu_df = seoul_loc_df[seoul_loc_df['do'] == '서울']
seoul_gu_df.head()

,docity,do,city,longitude,latitude
138,NaN,서울,강남구,127.049556,37.514575
139,NaN,서울,강동구,127.125864,37.527367
140,NaN,서울,강북구,127.027719,37.636956
141,NaN,서울,강서구,126.851675,37.548156
142,NaN,서울,관악구,126.953844,37.475386


In [47]:
seoul_gu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 138 to 162
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     0 non-null      float64
 1   do         25 non-null     object 
 2   city       25 non-null     object 
 3   longitude  25 non-null     float64
 4   latitude   25 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.2+ KB


In [63]:
seoul_gu_df.to_excel('./seoul_gu_info.xlsx', index=False)

##### 서울시 구별 스타벅스 매장수 구하기

In [49]:
sbuck_gu_counts = sbuck_df.pivot_table(index='구', 
                     values='매장명',
                     aggfunc='count').rename(columns={'매장명': '스타벅스_매장수'})

In [50]:
sbuck_gu_counts.head()

,스타벅스_매장수
구,
강남구,88
강동구,17
강북구,6
강서구,25
관악구,11


In [96]:
# seoul_gu_df 수정
seoul_gu_df = seoul_gu_df[['do', 'city', 'latitude', 'longitude']]
columns = {'do':'도', 'city':'구', 'latitude':'위도','longitude':'경도'}
seoul_gu_df.rename(columns=columns, inplace=True)
seoul_gu_df.reset_index(drop=True, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2552\938504763.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_gu_df.rename(columns=columns, inplace=True)


In [97]:
seoul_gu_df = seoul_gu_df[['도', '구', '경도', '위도']]


In [98]:
seoul_gu_df

,도,구,경도,위도
0,서울,강남구,127.049556,37.514575
1,서울,강동구,127.125864,37.527367
2,서울,강북구,127.027719,37.636956
3,서울,강서구,126.851675,37.548156
4,서울,관악구,126.953844,37.475386
5,서울,광진구,127.084533,37.535739
6,서울,구로구,126.889597,37.492650
7,서울,금천구,126.904197,37.449108
8,서울,노원구,127.058389,37.651461
9,서울,도봉구,127.049522,37.665833


In [99]:
seoul_sgg = pd.merge(seoul_gu_df, sbuck_gu_counts, how='left', on='구')
seoul_sgg

,도,구,경도,위도,스타벅스_매장수
0,서울,강남구,127.049556,37.514575,88
1,서울,강동구,127.125864,37.527367,17
2,서울,강북구,127.027719,37.636956,6
3,서울,강서구,126.851675,37.548156,25
4,서울,관악구,126.953844,37.475386,11
5,서울,광진구,127.084533,37.535739,18
6,서울,구로구,126.889597,37.492650,14
7,서울,금천구,126.904197,37.449108,12
8,서울,노원구,127.058389,37.651461,14
9,서울,도봉구,127.049522,37.665833,5


In [100]:
seoul_pop = pd.read_excel('./seoul_pop.xlsx')
seoul_pop.tail()

,구,계
20,관악구,502628
21,서초구,408979
22,강남구,537817
23,송파구,663704
24,강동구,464027


In [101]:
seoul_sgg = pd.merge(seoul_sgg, seoul_pop, how='left', on='구')
seoul_sgg

,도,구,경도,위도,스타벅스_매장수,계
0,서울,강남구,127.049556,37.514575,88,537817
1,서울,강동구,127.125864,37.527367,17,464027
2,서울,강북구,127.027719,37.636956,6,296934
3,서울,강서구,126.851675,37.548156,25,573711
4,서울,관악구,126.953844,37.475386,11,502628
5,서울,광진구,127.084533,37.535739,18,350925
6,서울,구로구,126.889597,37.492650,14,417983
7,서울,금천구,126.904197,37.449108,12,242467
8,서울,노원구,127.058389,37.651461,14,506989
9,서울,도봉구,127.049522,37.665833,5,312858


In [70]:
seoul_sgg = seoul_sgg[['도', '구', '경도', '위도', '스타벅스_매장수', '계_x']]
seoul_sgg.tail()

,도,구,경도,위도,스타벅스_매장수,계_x
20,서울,용산구,126.967522,37.536094,25,232482
21,서울,은평구,126.931242,37.599969,13,468766
22,서울,종로구,126.981642,37.570378,40,152212
23,서울,중구,126.999642,37.561003,53,131390
24,서울,중랑구,127.094778,37.603806,8,389928


In [72]:
# 사업체수
seoul_biz = pd.read_excel('./seoul_biz.xlsx')
seoul_biz.tail()

,구,사업체수,종사자수계
20,관악구,37915,128417
21,서초구,71958,487976
22,강남구,110007,801419
23,송파구,74531,400781
24,강동구,41660,157465


In [73]:
seoul_sgg_stat = pd.merge(seoul_sgg, seoul_biz, how='left', on='구') 
seoul_sgg_stat.tail()

,도,구,경도,위도,스타벅스_매장수,계_x,사업체수,종사자수계
20,서울,용산구,126.967522,37.536094,25,232482,29680,152605
21,서울,은평구,126.931242,37.599969,13,468766,36509,109031
22,서울,종로구,126.981642,37.570378,40,152212,48361,275063
23,서울,중구,126.999642,37.561003,53,131390,70308,386564
24,서울,중랑구,127.094778,37.603806,8,389928,39310,113144


In [74]:
columns = {'도':'도', '구':'구','경도':'경도','위도':'위도','스타벅스_매장수':'스타벅스_매장수','계_x':'인구수','사업체수':'사업체수','종사자수계':'종사자수'}

seoul_sgg_stat.rename(columns=columns,inplace=True)

In [ ]:
seoul_sgg_stat.to_excel('./seoul_sgg_stat.xlsx', index=False)
seoul_sgg_stat

,도,구,경도,위도,스타벅스_매장수,인구수,사업체수,종사자수
0,서울,강남구,127.049556,37.514575,88,537817,110007,801419
1,서울,강동구,127.125864,37.527367,17,464027,41660,157465
2,서울,강북구,127.027719,37.636956,6,296934,26385,80222
3,서울,강서구,126.851675,37.548156,25,573711,58806,290473
4,서울,관악구,126.953844,37.475386,11,502628,37915,128417
5,서울,광진구,127.084533,37.535739,18,350925,33706,129707
6,서울,구로구,126.889597,37.492650,14,417983,52791,240689
7,서울,금천구,126.904197,37.449108,12,242467,47964,255449
8,서울,노원구,127.058389,37.651461,14,506989,37909,133398
9,서울,도봉구,127.049522,37.665833,5,312858,25391,79097


##### 데이터 시각화

In [ ]:
sbuck_df.tail()

,매장명,위도,경도,매장타입,주소,구
595,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,중랑구
596,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동),중랑구
597,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",중랑구
598,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동),중랑구
599,중화역,37.601709,127.078411,general,서울특별시 중랑구 봉화산로 35,중랑구


In [ ]:
for idx in sbuck_df:
    print(idx)

매장명
위도
경도
매장타입
주소
구


In [ ]:
!pip install folium

                                              0.0/102.3 kB ? eta -:--:--
     -------------------------------------- 102.3/102.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
sbuck_map = folium.Map(location=[37.550823, 126.989502], tiles='Stamen Terrain', zoom_start=11)

for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']

    folium.CircleMarker(location=[lat,lng], fill=True, fill_color='green', fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=3).add_to(sbuck_map)
    
sbuck_map

In [ ]:
sbuck_map2 = folium.Map(location=[37.550823, 126.989502], tiles='Stamen Terrain', zoom_start=11)

for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']
    type = sbuck_df.loc[idx, '매장타입']

    # 매장타입별 색상 
    fillcolor = ''
    size = 2
    if type == 'general':
        fillcolor = 'gray'
        size = 1
    elif type == 'reserve':
        fillcolor = 'balck'
        size = 5
    elif type == 'generalDT':
        fillcolor = 'red'
        size = 3
    elif type == 'generalWT':
        fillcolor = 'blue'
        size = 3
    
    folium.CircleMarker(location=[lat,lng], fill=True, fill_color=fillcolor, fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=size).add_to(sbuck_map2)
    
sbuck_map2

In [ ]:
# 시군구경계표시
sgg_geojson_path = './seoul_geo_simple.json'
seoul_sgg_geo = json.load(open(sgg_geojson_path, encoding = 'utf-8'))

sbuck_map3 = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(seoul_sgg_geo, name='Seoul행정구역').add_to(sbuck_map3)
sbuck_map3

In [81]:
sbuck_mean = seoul_sgg_stat['스타벅스_매장수'].mean()
sbuck_mean

24.0

In [82]:
# 시군별 스타벅스 매장수를 버블지도로 시각화
sgg_geojson_path = './seoul_geo_simple.json'
seoul_sgg_geo = json.load(open(sgg_geojson_path, encoding = 'utf-8'))

sbuck_map3 = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(seoul_sgg_geo, name='Seoul행정구역').add_to(sbuck_map3)

for idx in seoul_sgg_stat.index:
    lat = seoul_sgg_stat.loc[idx, '위도']
    lng = seoul_sgg_stat.loc[idx, '경도']
    count = seoul_sgg_stat.loc[idx, '스타벅스_매장수']

    if count > sbuck_mean: # 구별 평균치보다 매장수가 많음
        fillcolor = 'red'
    else:
        fillcolor = 'blue'
    
    folium.CircleMarker(location=[lat,lng], color='#ffff00', fill_color=fillcolor, fill_opacity=0.7, weight=1.5, radius=count/2, popup=count).add_to(sbuck_map3)

sbuck_map3

In [83]:
seoul_sgg_geo = json.load(open(sgg_geojson_path, encoding='utf-8'))
seoul_sgg_geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.11879551821994, 37.557222485451305],
      [127.12146867175024, 37.55986003393365],
      [127.12435254630417, 37.56144246249796],
      [127.13593925898998, 37.56564793048277],
      [127.14930548011061, 37.56892250303897],
      [127.15511020940411, 37.57093642128295],
      [127.16683184366129, 37.57672487388627],
      [127.17038810813094, 37.576465605301046],
      [127.17607118428914, 37.57678573961056],
      [127.17905504160184, 37.57791388161732],
      [127.17747787800164, 37.57448983055031],
      [127.1781775408844, 37.571481967974336],
      [127.17995281860672, 37.569309661290504],
      [127.18122821955262, 37.56636089217979],
      [127.18169407550688, 37.56286338914073],
      [127.18408792

In [84]:
# 시군구경계표시
sgg_geojson_path = './seoul_geo_simple.json'
seoul_sgg_geo = json.load(open(sgg_geojson_path, encoding = 'utf-8'))

sbuck_map4 = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(seoul_sgg_geo, name='Seoul행정구역').add_to(sbuck_map4)

folium.Choropleth(
    geo_data=seoul_sgg_geo,
    data=seoul_sgg_stat,
    columns=['구', '스타벅스_매장수'],
    fill_color= 'Accent', #'YlGn', 'Spectral', 'RdYlGn', 'Accent', 'Reds', 'Greens', 'Blues', 'Oranges', 'PuOr'
    fill_opacity=0.7,
    line_opacity=0.5,
    key_on='properties.name'  # 이게 핵심 - key 값이 없으면 지도에 색상표시 못함
).add_to(sbuck_map4)    

sbuck_map4

In [75]:
seoul_sgg_stat

,도,구,경도,위도,스타벅스_매장수,인구수,사업체수,종사자수
0,서울,강남구,127.049556,37.514575,88,537817,110007,801419
1,서울,강동구,127.125864,37.527367,17,464027,41660,157465
2,서울,강북구,127.027719,37.636956,6,296934,26385,80222
3,서울,강서구,126.851675,37.548156,25,573711,58806,290473
4,서울,관악구,126.953844,37.475386,11,502628,37915,128417
5,서울,광진구,127.084533,37.535739,18,350925,33706,129707
6,서울,구로구,126.889597,37.492650,14,417983,52791,240689
7,서울,금천구,126.904197,37.449108,12,242467,47964,255449
8,서울,노원구,127.058389,37.651461,14,506989,37909,133398
9,서울,도봉구,127.049522,37.665833,5,312858,25391,79097


In [85]:
# 서울시 인구수별 분포
seoul_pop_choropleth = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=11)

folium.Choropleth(geo_data=seoul_sgg_geo, data=seoul_sgg_stat,
                  columns=['구', '인구수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(seoul_pop_choropleth)

seoul_pop_choropleth

- 결론 : 구에 살고있는 인구수와 스타벅스 매장수는 관계 거의 없다.

In [86]:
# 서울시 사업체수별 분포
seoul_biz_choropleth = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=11)

folium.Choropleth(geo_data=seoul_sgg_geo, data=seoul_sgg_stat,
                  columns=['구', '사업체수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(seoul_biz_choropleth)

seoul_biz_choropleth

- 결론 : 각 구별 사업체수와 스타벅스 매장수는 깊은 연관이 있다.

         동작구는 사당역 근처 리저브 매장을 폐점했다.(매출이 부진 예상)